In [68]:
# from transformers import pipeline
# from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import time
import os
from mistralai import Mistral
import pandas as pd
import sys
import warnings
import csv
from requests import request
import json
from datetime import datetime
import torch
from collections import deque
import math
import heapq

In [69]:
# !pip install openai
# !pip install langchain
# !pip install langchain_community
# !pip install -U langchain-openai
# !pip install mistralai

In [75]:
def get_chat_template(user_message: str, instructions: str) -> str:
  prompt = ""
  prompt = prompt + f"""<s>[INST] {instructions}\n\n{user_message}[/INST]"""
  return prompt

In [76]:
def getLLMResponse(instructions, user_query):
  model = "mistral-large-latest"
  client = Mistral(api_key="gDSJrKmxHmIWvfTVXnUxucyK4GJbgSiV")
  conversation = get_chat_template(user_query,instructions)
  chat_response = client.chat.complete(
      model= model,
      temperature=0.001,
      messages = [
          {
              "role": "user",
              "content": conversation,
          },
      ]
  )
  res = chat_response.choices[0].message.content
  return res

In [94]:

thought_generation_prompts = [
"You are an intelligent complaint-handling assistant. You are given a [INPUT] which is a complaint that the complainant is trying to address. The complaints are against regulated entities. \n###Your Task:\n1. Carefully analyze the [INPUT] and [LIST OF REASONS] to identify whether complainant has filed the complaint with the entity first before filing it with you. Your reasonings should be derived only from information available in the Complaint. \n2. Output only the next {number_of_samples} versions of reasoning as the next entry in the list indicating whether complainant has already filed the complaint with the entity or not based on your deduction.\n\n### Based on your analysis of the Complaint and List of Reasons, assign **one of the possible TAGS** : TAG `Definitely Filed` - The complainant **explicitly states** or in affirmation to have filed the complaint with the entity, regardless of whether it was rejected or still awaiting a response from the entity. \n TAG `Probably Filed` - If the complainant **confirms** that the complaint is already filed or registered with the entity before asking for your assitance. \n TAG `Probably Not Filed` - If the complainant **intends to file a complaint** but there **strictly no evidence about filing the complaint** with the entity before taking assistance from you. \n TAG `Definitely Not Filed` - If the complainant only mentions distress regarding deficiency in service, threats received by the entity without approaching the entity or specifically mentions that they have not filed a complaint with the entity yet. \n\n### Note - Always consider:\n1. Provide **exactly {number_of_samples} ** well-reasoned justifications supporting the assigned classification.\n2. Keep your deduction crisp and to the point.\n3. The reasons must be strictly **based on explicit statements in the Complaint and List of Reasons** and should **never introduce any assumptions or implications**.\n3. A TAG can be used for multiple reasons as well.\n4. **Never infer or assume about complaint being already filed if not confirmed or explictly stated by the complainant**.\n5. Re-think if the complainant has stated that they have filed the complaint or they are waiting for reply from the entity?\n6. The output list must have {number_of_samples} TAG and its corresponding Reason in the format [('TAG', 'Reason'), ('TAG', 'REASON'), ('TAG', 'REASON')]. STRICTLY FOLLOW THIS **FORMAT** AS OUTPUT WITH COMPLETE QUOTATION MARKS AND BRACKETS.\n\n### Here is the Input and Reasons:\n\n"
,"You are an intelligent complaint-handling assistant. You are given a [INPUT] which is a complaint that the complainant is trying to address. The complaints are against regulated entities. Your task is to evaluate the given Complaint and figure out if the complaint was either **rejected by the entity**, the complainant is **not satisfied with the resolution** provided by the entity or the entity has **delayed responding back** to the complainant. \n###Your Task:\n1. Carefully analyze the `List of Reasons` to identify whether the complaint was either rejected by the entity or the complainant is not satisfied with the resolution provided by the entity or the entity has not responded to the complainant regarding the complaint. Your reasonings should be derived only from information available in the Complaint. \n2. Output only the next {number_of_samples} different versions of reasoning as the next entry in the list indicating whether or not the complaint is either rejected, or received unsatisfactory response or is pending to be acknowledged by the entity based on your deduction.\n\n### Based on your analysis of the Complaint and list of reasons, assign **one of the possible TAGS**: TAG `Rejected` - If the complainant has explicitly stated that the complaint was **rejected** or **denied** by the entity. \n TAG `Unsatisfactory` - If the complainant has explicitly stated that a **response or resolution is received from the entity** but the complainant is still not satisfied with the resolution. \n TAG `Delayed` - If the entity has **not responded** or **Delayed** responding to the complainant despite attempts by the complainant in seeking a resolution.\n\n### Note - Always consider:\n1. Never output more than {number_of_samples} versions of reasoning.\n2. Keep your deduction crisp and to the point.\n3. A TAG can be used for multiple reasons as well.\n4. If there is no mention of entity acknowledging the complaint (either Rejected or Unsatisfactory response), then always TAG complaint as **Delayed** without any assumptions.\n5. The output list must have {number_of_samples} TAG and its corresponding Reason in the format [('TAG', 'Reason'), ('TAG', 'REASON'), ('TAG', 'REASON')]. STRICTLY FOLLOW THIS FORMAT AS OUTPUT WITH COMPLETE QUOTATION MARKS AND BRACKETS.\n\n### Here is the Input and Reasons:\n\n"
,"You are an intelligent complaint-handling assistant. You are given a [INPUT] which is a complaint that the complainant is trying to address. The complaints are against regulated entities. Your task is to evaluate the given complaint and determine whether it has been pending a response from the regulated entity for **more than 30 days**. \n###Your Task:\n1. Analyze the [INPUT] and [LIST OF REASONS] to identify whether it has been more than 30 days since the complaint was filed with the entity. Your reasonings should be derived only from information available in the complaint. \n2. Output only the next {number_of_samples} versions of reasoning as the next entry in the list indicating whether or not it has been more than 30 days since the complainant has received a response from the entity.\n\n### Based on your analysis of the Complaint and list of reasons, assign either **one of the possible TAGS**: TAG `More than 30 Days` - The complainant **explicitly states** or in affirmation that complaint has been pending for more than 30 days (e.g., 'Last Month', '40 days passed'). \n TAG `Within 30 Days` - The complaint explicitly states that it was filed within 30 days (e.g., 'Last Week', '20 days ago', 'yesterday','more than 15 days'). Any complainant which is pending for more than 1 days and less than 30 days will always be assigned the TAG, `Within 30 Days`.\n\n### Note - Always consider:\n1. Never output more than {number_of_samples} versions for the next entry in list.\n2. Keep your deduction crisp and to the point.\n3. A TAG can be used for multiple reasons as well.\n4. Never assume more than 30 days unless the complaint explicitly states so.\n5. **If no time duration is mentioned, assume it is within 30 days**.\n6. The output list must have {number_of_samples} TAG and its corresponding Reason in the format [('TAG', 'Reason'), ('TAG', 'REASON'), ('TAG', 'REASON')]. STRICTLY FOLLOW THIS FORMAT AS OUTPUT WITH COMPLETE QUOTATION MARKS AND BRACKETS.\n\n### Here is the Input and Reasons:\n\n"
]
evaluation_prompts = [
"You are an intelligent complaint-handling evaluator. Your role is to **strictly** assign a negative or positive score between the range **-10 to 10** to establish certainty on the list of reasonings provided for complaint's filing status using [INPUT] as complaint. The complaint is by a citizen seeking resolution against their issue with unsatisfactory service, or response from a regulated entity (RE). Strictly follow the Task:\n\n### Task- Analyze the `list of reasons` and `complaint` to determine whether the complainant has filed a complaint with the entity.\n\n### ### Scoring Criteria:\n**10**: You are absolutely certain that the complaint was 'Definitely Filed'.\n**1 to 9**: The complaint was 'Probably Filed', but certainty decreases as the score moves toward 1.\n**-9 to 0**: The complaint was 'Probably Not Filed', but certainty decreases as the score moves toward 0.\n**-10**: You are absolutely certain that the complaint was 'Definitely Not Filed'.\n\n### Notes-\n1. Always output a single `positive` or `negative` number as score.\n2. The output must never contain anything (explanation, text, spaces, special characters or punctuations) other than just a positive or negative number (score).\n3. Always follow the `Scoring Criteria` and if the query only expresses the 'want' of filing a complaint rather than a confirmation of complaint being already filed, then always provide a **negative score**.\n4. Never provide score equal to 10 or -10 for TAGs containing `Probably` and recheck the `Scoring Criteria`.\n5. STRICTLY FOLLOW THIS FORMAT ONLY FOR OUTPUT -  output: SCORE. \n\n### Here is the complaint and list of reasons:\n\n"
,"You are an intelligent complaint handling evaluator. Your role is to Strictly provide a negative or positive score between the range -10 to 10 to establish certainty on the list of reasonings provided for complaint which is [INPUT] and list of reasons. The complaint is by a citizen seeking resolution against their issue with unsatisfactory service, or response from a regulated entity. Strictly follow the Task:\n\n### Task- Analyze the `list of reasons` and `complaint` to deduce whether the entity rejected the complaint, complainant is not satisfied with the resolution or the response from the entity is delayed. \n\n### Scoring Criteria:\n**10**: You are **completely certain** that the entity has definitely **Delayed** responding or acknowledging back to the complainant.\n**1 to 9**: You believe that the entity has likely **Delayed or not responded to the complainant**, but with decreasing certainty as the score approaches 0.\n**-9 to 0**: You are certain that the complaint **was **rejected** by the entity and complainant is not satisfied with the entity's response** related to the complaint , but with decreasing certainty as the score approaches 0.\n**-10**: You are completely certain that the complaint **was rejected by the bank**or the complainant is **not satisfied with the resolution provided by the entity**.;\n\n### Notes-\n1. Always output a single `positive` or `negative` score.\n2. The output must never contain anything (explanation, text, spaces, special characters or punctuations) other than just a positive or negative number (score).\n3. Always follow the `Scoring Criteria` and make sure if the TAG is `Delayed`, then the score will always be **positive**.\n4. STRICTLY FOLLOW THIS FORMAT ONLY FOR OUTPUT -  output: SCORE. \n\n### Here is the input and list of reasons:\n\n"
,"You are an intelligent complaint-handling evaluator. Your role is to **strictly** provide a negative or positive score between the range -10 to 10 to establish certainty on the list of reasonings provided for complaint's resolution status by the entity. The [INPUT] which is complaint by a citizen seeking resolution against their issue with unsatisfactory service, or response from a regulated entity. Your task is to determine whether the complaint has **explicitly** been pending for more than 30 days. Strictly follow the Task:\n\n### Task- Analyze the `list of reasons` and `complaint` to deduce whether *it has been more than 30 days since the complaint was filed?*. \n\n### Scoring Criteria:\n**10**: The complainant explicitly states that it has been **More than 30 Days** since the entity responded back.\n**-10**: the complaint explicitly states that it was filed `within last 30 days` (e.g., Last week, 5 days ago, more than 15 days) OR does not mention any duration confirming more than 30 days.\n\n### Notes-\n1. Always output a single `positive` or `negative` score.\n2. The output must never contain anything (explanation, text, spaces, special characters or punctuations) other than just a positive or negative number (score).\n3.Strictly analyze and compare the date mentioned in the Complaint and follow the `Scoring Criteria` always.\n\n4. STRICTLY FOLLOW THIS FORMAT ONLY FOR OUTPUT -  output: SCORE. \n\n### Here is the Complaint and list of reasons:\n\n"
]

In [97]:
def thought_generation(state, prompt, number_of_samples):
    return getLLMResponse(state, thought_generation_prompts[prompt].format(number_of_samples=number_of_samples)).split(":")[:number_of_samples]


def evaluation(state, prompt):
    ans = getLLMResponse(state, evaluation_prompts[prompt])
    return ans

def tree_of_thoughts_bfs(query, max_depth, threshold, number_of_samples, number_of_questions):
  res = []
  for i in range(number_of_questions):
      s = deque([ { "query":query, "list":[], "score": 0 } ])
      max_element = s[0]
      while s:
          x = s.popleft()
          print(x)
          if abs(x["score"]) > abs(max_element["score"]):
              max_element = x.copy()
          print("len(x[list]): ", len(x["list"]), ", Max depth: ", max_depth)
          if len(x["list"]) < max_depth:
              candidates = thought_generation( "[INPUT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]) , i, number_of_samples)
              for candidate in candidates:
                  try:
                      score = evaluation("[INPUT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]+[candidate]), i)
                      print("Score:", score)
                      score = float(score)
                  except:
                    score = -5
                  if abs(score) >= threshold:
                      s.append({ "query":query, "list":x["list"]+[candidate], "score": score})
      res.append(max_element.copy())
  return res

def tree_of_thoughts_bfs_heirarchial(query, max_depth, threshold, number_of_samples, number_of_questions):
    res = []
    # for i in range(number_of_questions):
    s = deque([ { "query":query, "list":[], "score": 0, "level": 0 } ])
    max_element = s[0]
    while s:
        x = s.popleft()
        print(x)
        if abs(x["score"]) > abs(max_element["score"]):
            max_element = x.copy()
        print("len(x[list]): ", len(x["list"]), ", Max depth: ", max_depth)
        if len(x["list"]) < max_depth:
            candidates = thought_generation( "[COMPLAINT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]) , x["level"], number_of_samples)
            for candidate in candidates:
                try:
                    score = evaluation("[COMPLAINT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]+[candidate]), x["level"])
                    score = float(score)
                except:
                    score = extract_number(evaluation("[INPUT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]+[candidate]), x["level"]))
                    print("except score:",score)
                    if len(score) != 0:
                        print("except score:",float(score[0]))
                        score = float(score[0])
                    else:
                        score = -5
                if score>0 and x["level"]<number_of_questions:
                    s.append({ "query":query, "list":[], "score": score, "level": x["level"]+1 })
                elif abs(score) >= threshold:
                        s.append({ "query":query, "list":x["list"]+[candidate], "score": score, "level": x["level"]})

    res.append(max_element.copy())
    return res

def tree_of_thoughts_dfs(query, max_depth, threshold, number_of_samples, number_of_questions):
    res = []
    for i in range(number_of_questions):
        s = deque([ { "query":query, "list":[], "score": 0 } ])
        max_element = s[0]
        while s:
            x = s.pop()
            if abs(x["score"]) > abs(max_element["score"]):
                max_element = x.copy()
            if len(x["list"]) < max_depth:
                candidates = thought_generation( "[COMPLAINT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]) , i, number_of_samples)
                for candidate in candidates:
                    score = evaluation("[COMPLAINT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]+[candidate]), i)
                    if abs(score) >= threshold:
                        s.append({ "query":query, "list":x["list"]+[candidate], "score": score })
        res.append(max_element.copy())
    return res

def tree_of_thoughts_astar(query, max_depth, threshold, number_of_samples, number_of_questions):
    res = []
    for i in range(number_of_questions):
        dict_container = []
        dict_container.append({ "query":query, "list":[], "g":0, "h":0, "score": 10 })
        s=[]
        heapq.heappush(s, (10, 0) )
        max_element = { "query":query, "list":[], "g":0, "h":0, "score": 10 }
        while s:
            x = dict_container[heapq.heappop(s)[1]]
            # print(x)
            if x["score"] < max_element["score"]:
                max_element = x.copy()
            if len(x["list"]) < max_depth:
                candidates = thought_generation( "[COMPLAINT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]) , i, number_of_samples)
                for candidate in candidates:
                    c_g=x["g"]+0.5
                    c_h=evaluation("[COMPLAINT] - " + x["query"] + " [LIST OF REASONS] - " + " ".join(x["list"]+[candidate]), i)
                    c_f=10*math.tanh(c_g+10-abs(c_h))
                    dict_container.append({ "query":query, "list":x["list"]+[candidate], "g":c_g, "h":c_h, "score": c_f })
                    heapq.heappush(s, (c_f, len(dict_container)-1 ))
        res.append(max_element.copy())
    return res


def get_Judge_Tree_of_Thoughts(query, thought_method):
    MAX_DEPTH = 2
    THRESHOLD = 5
    NUMBER_OF_SAMPLES = 3
    NUMBER_OF_QUESTIONS=len(thought_generation_prompts)

    if thought_method == "dfs":
        result_final = tree_of_thoughts_dfs(query, max_depth=MAX_DEPTH, threshold=THRESHOLD, number_of_samples=NUMBER_OF_SAMPLES, number_of_questions=NUMBER_OF_QUESTIONS)
        dfs_result_final = result_final
        print("RESULT DFS: ", result_final)

    elif thought_method == "bfs":
        result_final = tree_of_thoughts_bfs(query, max_depth=MAX_DEPTH, threshold=THRESHOLD,number_of_samples=NUMBER_OF_SAMPLES, number_of_questions=NUMBER_OF_QUESTIONS)
        bfs_result_final = result_final
        print("RESULT BFS: ", result_final)
        return(result_final)

    elif thought_method == "bfs-h":
        result_final = tree_of_thoughts_bfs_heirarchial(query, max_depth=MAX_DEPTH, threshold=THRESHOLD,number_of_samples=NUMBER_OF_SAMPLES, number_of_questions=NUMBER_OF_QUESTIONS)
        # bfs_result_final = tree_of_thoughts_bfs_heirarchial(query, max_depth=MAX_DEPTH, threshold=THRESHOLD, number_of_samples=NUMBER_OF_SAMPLES, number_of_questions=NUMBER_OF_QUESTIONS)
        print("RESULT BFS: ", result_final)
        return(result_final)

    elif thought_method == "astar":
        result_final = tree_of_thoughts_astar(query, max_depth=MAX_DEPTH, threshold=THRESHOLD,number_of_samples=NUMBER_OF_SAMPLES, number_of_questions=NUMBER_OF_QUESTIONS)
        astar_result_final = result_final
        print("RESULT Astar: ", result_final)

    Complaint_filed_score = float(result_final[0]["score"])
    Complaint_rejected_score = float(result_final[1]["score"])
    response_from_re_score = float(result_final[2]["score"])

    if Complaint_filed_score:
      if Complaint_filed_score > 0:
          if Complaint_rejected_score > 0:
              if response_from_re_score > 0:
                return "I suggest you to file a complaint immediately"
              else:
                return "Please wait for your RE to respond back"
          else:
            return "If you are not satisfied with RE's response, then immidiately call for help"
      else:
        return "Please file your complaint with your RE first"
    else:
      return "Recommending to file the complaint asap."


In [98]:
query = "I want to file a complaint"
thought_method = "bfs"
answer = get_Judge_Tree_of_Thoughts(query, thought_method)

{'query': 'I want to file a complaint', 'list': [], 'score': 0}
len(x[list]):  0 , Max depth:  2
Score: ```
[INPUT] - I want to file a complaint
[LIST OF REASONS] -
1. I have been trying to reach customer service for a week with no response.
2. The product I received is defective and they are not replacing it.
3. I have sent multiple emails but have not received any acknowledgment.
```

output: -5
Score: -5
Score: -5
{'query': 'I want to file a complaint', 'list': ['### Input'], 'score': -5}
len(x[list]):  1 , Max depth:  2
Score: -5
{'query': 'I want to file a complaint', 'list': ['\nI have been facing issues with my bank for the past few months. They have been charging me hidden fees, and their customer service is terrible. I have tried to resolve this issue multiple times, but they keep giving me the runaround. I am fed up and want to file a complaint.\n\n### List of Reasons'], 'score': -5.0}
len(x[list]):  1 , Max depth:  2
Score: Output: -5
{'query': 'I want to file a complaint', 